In [66]:
import warnings
warnings.filterwarnings("ignore")

In [67]:
import re

delimiter = '\n'
int_regex = re.compile(r'^[0-9]+$')
float_regex = re.compile(r'^[0-9]+\.[0-9]+$')
string_regex = re.compile(r'^\"[a-zA-Z0-9\ ]*\"$')

position_regex_integers = re.compile(r"^[0-9]{0,5}$")
position_regex_strings = re.compile(r"^[0-9a-zA-Z]{0,5}$")

valor = '"OLA MUNDO"'

if string_regex.match(valor) == None:
    print('Not a position')
else:
    print('position')

position


In [68]:
import pandas as pd

variable_table = pd.DataFrame(columns = ['label', 'type', 'value'])

In [69]:
palavras_reservadas = ['PRINT', 'DEFINE', 'SET', 'MOVE', 'DELAY', 'OPEN', 'CLOSE']

In [70]:
with open('ACL_program.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()
    lista = list(map(lambda x: x.replace('\n', ''), lines))

In [71]:
def SET(args):
    global variable_table
    value = args.split('=')[1]
    label = args.split('=')[0]

    type = variable_table.loc[variable_table['label'] == label]['type'].values[0]

    if type == None:
        return 'ERRO SET'
    
    elif type != 'Unknown':
        if type == 'int':
            if int_regex.match(value) != None:
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)
            else:
                return 'ERRO SET'
        
        elif type == 'float':
            if float_regex.match(value) != None:
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)
            else:
                return 'ERRO SET'

        elif type == 'string':
            if string_regex.match(value) != None:
                value = value[0:10]
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)
            else:
                return 'ERRO SET'

    else:
            if int_regex.match(value) != None:
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'type'] = 'int'
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)
        
            elif float_regex.match(value) != None:
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'type'] = 'float'
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)

            elif string_regex.match(value) != None:
                value = value[0:10]
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'type'] = 'string'
                variable_table.at[variable_table.loc[variable_table['label'] == label].index.values[0], 'value'] = value
                return 'SET {}={}'.format(label, value)
            else:
                return 'ERRO SET'

In [72]:
def DEFINE(args):
    global variable_table
    if '=' in args:      
        value = args.split('=')[1]
        label = args.split('=')[0]
        if int_regex.match(value) != None:
            variable_table = variable_table.append(pd.DataFrame({'label':[label], 'type':['int'], 'value': None}), ignore_index= True)
            return ['DEFINE {}'.format(label), SET(args)]

        elif float_regex.match(value) != None:
            variable_table = variable_table.append(pd.DataFrame({'label':[label], 'type':['float'], 'value': None}), ignore_index= True)
            return ['DEFINE {}'.format(label), SET(args)]

        elif string_regex.match(value) != None:
            variable_table = variable_table.append(pd.DataFrame({'label':[label], 'type':['string'], 'value': None}), ignore_index= True)
            return ['DEFINE {}'.format(label), SET(args)]
        else:
            return 'ERRO DEFINE'
    elif args != None and type(args) == str:
        variable_table = variable_table.append(pd.DataFrame({'label':[args], 'type':['Unknown'], 'value': None}), ignore_index= True)
        return 'DEFINE {}'.format(args)
        
    else:
        return 'ERRO DEFINE'

In [74]:
def PRINT(args):
    if string_regex.match(value) != None:
        value = value[0,10]
        return 'PRINT {}'.format(args)
    elif variable_table.loc[variable_table['label'] == args]['label'].values[0] != None:
        return 'PRINT {}'.format(args)
    else:
        return "ERRO PRINT"


In [75]:
def MOVE(args):
    if position_regex_integers.match(args) != None:
        return 'MOVE {}'.format(args)
    elif position_regex_strings.match(args) != None:
        return 'MOVE {}'.format(args)
    else:
        return "ERRO MOVE"

In [76]:
transpiled_code = []
for line in lista:
    print(line)
    if line == '':
        continue
    elif line.split(' ')[0] not in palavras_reservadas:
        transpiled_code.append('ERRO')
        break
        
    else:
        function = line.split(' ')[0]

        if function == 'DEFINE':
            retorno = DEFINE(line.split(' ')[1])
            transpiled_code.append(retorno)
            if retorno == 'ERRO DEFINE' or retorno == 'ERRO SET':
                break

        elif function == 'PRINT':
            retorno = PRINT(line.split(' ')[1])
            transpiled_code.append(retorno)
            if retorno == 'ERRO PRINT':
                break
        
        elif function == 'MOVE':
            retorno = MOVE(line.split(' ')[1])
            transpiled_code.append(retorno)
            if retorno == 'ERRO MOVE':
                break
        elif function == 'SET':
            retorno = SET(line.split(' ')[1])
            transpiled_code.append(retorno)
            if retorno == 'ERRO SET':
                break

print(transpiled_code)

DEFINE STR="OLAMUNDO"
DEFINE INT=12
DEFINE FLOAT=12.02

DEFINE VAR
SET VAR=25

SET VAR="STR"
[['DEFINE STR', 'SET STR="OLAMUNDO"'], ['DEFINE INT', 'SET INT=12'], ['DEFINE FLOAT', 'SET FLOAT=12.02'], 'DEFINE VAR', 'SET VAR=25', 'ERRO SET']


In [77]:
variable_table

,label,type,value
0,STR,string,"""OLAMUNDO"""
1,INT,int,12
2,FLOAT,float,12.02
3,VAR,int,25
